markstro
3/27/2020

compute the solar table for the HRUs for each day of the year

In [6]:
import numpy as np
#import pandas as pd
import fiona 
import geopandas as gpd
#import matplotlib.pyplot as plt
#%matplotlib inline
#from matplotlib import cm
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# DANGER!  These paths are pointing to the GF 1.0 version of the parameter files until I get the new ones!

In [38]:
hru_aspect_fn = 'c:/Users/markstro/work/input/hru_aspect.csv'
hru_slope_fn = 'c:/Users/markstro/work/input/hru_slope.csv'
hru_lat_fn = 'c:/Users/markstro/work/input/hru_lat.csv'

In [30]:
eccentricity = 0.01671
daysyr = 365.242
degday = 360.0 / daysyr
radians = np.pi / 180.0
degdayrad = degday * radians

In [48]:
obliquity = np.zeros(366)
solar_declination = np.zeros(366)

for jday in range(1,367):
    jday_f = float(jday)
    
    obliquity[jday-1] = 1.0 - (eccentricity * np.cos(jday_f - 3.0) * degdayrad)
    
    y = degdayrad * (jday_f - 1.0)
    y2 = 2.0 * y
    y3 = 3.0 * y
    solar_declination[jday-1] = 0.006918 - 0.399912 * np.cos(y) + 0.070257 * np.sin(y) \
                                - 0.006758 * np.cos(y2) + 0.000907 * np.sin(y2) \
                                - 0.002697 * np.cos(y3) + 0.00148 * np.sin(y3)
    
    #print(jday, obliquity[jday-1], solar_declination[jday-1])

In [39]:
hru_aspect = np.genfromtxt(hru_aspect_fn, skip_header=True)
print(hru_aspect)

hru_slope = np.genfromtxt(hru_slope_fn, skip_header=True)
print(hru_slope)

hru_lat = np.genfromtxt(hru_lat_fn, skip_header=True)
print(hru_lat)

[155.877652  64.640189 225.288905 ... 288.246064 279.472498 286.836972]
[0.066516 0.045131 0.042507 ... 0.074963 0.227312 0.272438]
[46.2062   45.609654 45.629805 ... 40.972683 41.124957 41.413014]


In [55]:
#  This is the sunrise equation
#  Lat is the latitude
#  Solar_declination is the declination of the sun on a day
#  T is the angle hour from the local meridian (local solar noon) to the
#  sunrise (negative) or sunset (positive).  The Earth rotates at the angular
#  speed of 15 degrees/hour (2 pi / 24 hour in radians) and, therefore, T/15 degress (T*24/pi
#  in radians) gives the time of sunrise as the number of hours before the local
#  noon, or the time of sunset as the number of hours after the local noon.
#  Here the term local noon indicates the local time when the sun is exactly to
#  the south or north or exactly overhead.
#
#  This is vectorize and runs over all of the HRUs in one call

def compute_t(Lat, Solar_declination):
#    if tx < -1.0:
#        T = np.pi
#    elif tx > 1.0:
#        T = 0.0
#    else:
#        T = np.arccos(tx)
        
    tx = -np.tan(Lat) * np.tan(Solar_declination)
    T = np.arccos(tx)
    T = np.where(tx < -1.0, np.pi, T)
    T = np.where(tx > 1.0, 0.0, T)

    return T

In [61]:
# These np functions are vectorized, all variables are np arrays

sl = np.arctan(hru_slope)
cossl = np.cos(sl)
a = radians * hru_aspect

# x0 latitude of HRUs
x0 = hru_lat * radians

# x1 latitude of equivalent slope
x1 = np.arcsin(cossl * np.sin(x0) + np.sin(sl) * np.cos(x0) * np.cos(a))

# d1 is the denominator of equation 12, Lee, 1963
d1 = cossl * np.cos(x0) - np.sin(sl) * np.sin(x0) * np.cos(a)

# x2 is the difference in longitude between the location of
# the HRU and the equivalent horizontal surface expressed in angle hour
# This is equation 12 from Lee, 1963
#x2 = ATAN(SIN(sl)*SIN(a)/d1)
x2 = np.arctan(np.sin(sl) * np.sin(a) / d1)

#      IF ( d1<0.0D0 ) x2 = x2 + PI
x2 = np.where(d1 >= 0, x2, x2 + np.pi) 

# r0 is the minute solar constant cal/cm2/min
r0 = 2.0
# r0 could be 1.95 (Drummond, et al 1968)

        
#print(sl)
#print(cossl)
#print(a)
#print(x0)
#print(x1)
#print(d1)
#print(min(d1))
#print(x2)
#print(max(x2))

#for ihru in range(nhru):
#            CALL compute_soltab(obliquity, Solar_declination, Hru_slope(n), Hru_aspect(n), &
#     &                      Hru_lat(n), Hru_cossl(n), Soltab_potsw(1, n), &
#     &                      Soltab_sunhrs(1, n), Hru_type(n), n)

#for jday in range(1,367):
for jday in range(1,2):

#    jday_f = float(jday)
    d = solar_declination[jday-1]
    print(jday, d)
    
# This is adjusted to express the variability of available insolation as
# a function of the earth-sun distance.  Lee, 1963, p 16.
# r1 is the hour solar constant cal/cm2/hour
# r0 is the minute solar constant cal/cm2/min
# 60.0D0 is minutes in an hour
# Obliquity is the obliquity of the ellipse of the earth's orbit around the sun. E
# is also called the radius vector of the sun (or earth) and is the ratio of
# the earth-sun distance on a day to the mean earth-sun distance.
# obliquity = ~23.439 (obliquity of sun)
    r1 = 60.0 * r0 / (obliquity[jday-1] * obliquity[jday-1])
    print(jday, r1)
    
#  compute_t is the sunrise equation.
#  t7 is the hour angle of sunset on the equivalent slope
#  t6 is the hour angle of sunrise on the equivalent slope
    t = compute_t(x1, d)
    t7 = t - x2
    t6 = -t - x2
    print(jday, t7)
    print(jday, t6)
    
#  compute_t is the sunrise equation.
#  t1 is the hour angle of sunset on a hroizontal surface at the HRU
#  t0 is the hour angle of sunrise on a hroizontal surface at the HRU
    t = compute_t(x1, d)
    t1 = t
    t0 = -t
    print(jday, t1)
    print(jday, t0)
    
# For HRUs that have an east or west direction component to their aspect, the
# longitude adjustment (moving the effective slope east or west) will cause either:
# (1) sunrise to be earlier than at the horizontal plane at the HRU
# (2) sunset to be later than at the horizontal plane at the HRU
# This is not possible. The if statements below check for this and adjust the
# sunrise/sunset angle hours on the equivalent slopes as necessary.
#
# t3 is the hour angle of sunrise on the slope at the HRU
# t2 is the hour angle of sunset on the slope at the HRU
#        IF ( t7>t1 ) THEN
#          t3 = t1
#        ELSE
#          t3 = t7
#        ENDIF
    t3 = np.where(t7 < t1, t1, t7) 
        
#        IF ( t6<t0 ) THEN
#          t2 = t0
#        ELSE
#          t2 = t6
#        ENDIF
    t2 = np.where(t6 < t0, t0, t6) 
    
    print(jday, t3)
    print(jday, t2)
    

1 -0.402449
1 119.97129513412267
1 [1.12999811 1.04328934 1.19075429 ... 1.27068985 1.47629691 1.5039665 ]
1 [-1.20384607 -1.16209413 -1.10696965 ... -1.07876954 -0.87928498
 -0.78869154]
1 [1.16692209 1.10269173 1.14886197 ... 1.17472969 1.17779094 1.14632902]
1 [-1.16692209 -1.10269173 -1.14886197 ... -1.17472969 -1.17779094
 -1.14632902]
1 [1.16692209 1.10269173 1.19075429 ... 1.27068985 1.47629691 1.5039665 ]
1 [-1.16692209 -1.10269173 -1.10696965 ... -1.07876954 -0.87928498
 -0.78869154]


<ipython-input-55-ecf22fe4c050>:23: RuntimeWarning: invalid value encountered in arccos
  T = np.arccos(tx)
